# Amazon SageMaker で PyTorch の GNN を使ったノード分類を行う
このサンプルノートブックは、[PyTorch geometric のサンプルコード](https://pytorch-geometric.readthedocs.io/en/latest/notes/colabs.html)を参考にしました。

## Node Classification with Graph Neural Networks

[Previous: Introduction: Hands-on Graph Neural Networks](https://colab.research.google.com/drive/1h3-vJGRVloF5zStxL5I0rSy4ZUPNsjy8)

This tutorial will teach you how to apply **Graph Neural Networks (GNNs) to the task of node classification**.
Here, we are given the ground-truth labels of only a small subset of nodes, and want to infer the labels for all the remaining nodes (*transductive learning*).

To demonstrate, we make use of the `Cora` dataset, which is a **citation network** where nodes represent documents.
Each node is described by a 1433-dimensional bag-of-words feature vector.
Two documents are connected if there exists a citation link between them.
The task is to infer the category of each document (7 in total).

This dataset was first introduced by [Yang et al. (2016)](https://arxiv.org/abs/1603.08861) as one of the datasets of the `Planetoid` benchmark suite.
We again can make use [PyTorch Geometric](https://github.com/rusty1s/pytorch_geometric) for an easy access to this dataset via [`torch_geometric.datasets.Planetoid`](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.Planetoid):


## 準備

**このサンプルでは、カスタムコンテナを Amazon ECR に push する必要があります。**以下の操作でこのノートブックインスタンスで使用している IAM ロールに Amazon ECR にイメージを push するための権限を追加してください。

1. Amazon SageMaker コンソールからこのノートブックインスタンスの詳細画面を表示<br>（左側のメニューのインスタンス -> ノートブックインスタンス -> インスタンス名をクリック）
1. 「アクセス許可と暗号化」の「IAM ロール ARN」のリンクをクリック（IAM のコンソールに遷移します）
1. 「ポリシーをアタッチします」と書いてある青いボタンをクリック
1. 検索ボックスに ec2containerregistry と入力し  AmazonEC2ContainerRegistryFullAccess のチェックボックスをチェックする
1. 「ポリシーのアタッチ」と書いてある青いボタンをクリック

以下のセルでは、Amazon SageMaker を使うためのセットアップを行います。ロールの情報、ノートブックインスタンスのリージョン、アカウントID などの情報を取得しています。

In [ ]:
import boto3
import sys
import sagemaker
import numpy as np
from sagemaker import get_execution_role

role = get_execution_role()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
session = sagemaker.Session()
bucket = session.default_bucket()
s3_output = session.default_bucket()
s3_prefix = 'gnn-byo'

In [ ]:
!mkdir docker
!mkdir docker/processing
!mkdir docker/train
!mkdir docker/inference

In [ ]:
%%writefile docker/processing/requirements.txt

boto3==1.17.35
torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html
torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html
torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html
torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html
torch-geometric==1.6.3
matplotlib==3.3.4
scikit-learn==0.24.1


In [ ]:
!cp docker/processing/requirements.txt docker/train/requirements.txt

## Amazon SageMaker Experiments のセットアップ
Amazon SageMaker Experiments のライブラリをインストールします。

In [ ]:
!{sys.executable} -m pip install sagemaker-experiments requests

前処理用、学習用の Expetiments を作成します。

In [ ]:
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker
import time

gnn_experiment_preprocess = Experiment.create(
    experiment_name=f"gnn-byo-preprocess-{int(time.time())}", 
    description="node classification using gnn (preprocess)", 
    sagemaker_boto_client=boto3.client('sagemaker'))
print(gnn_experiment_preprocess)

gnn_experiment_train = Experiment.create(
    experiment_name=f"gnn-byo-train-{int(time.time())}", 
    description="node classification using gnn (train)", 
    sagemaker_boto_client=boto3.client('sagemaker'))
print(gnn_experiment_train)

このサンプルノートブックでは、データの前処理、前処理したデータを使ってモデルの学習、学習済みモデルを使ってバッチ推論、の順でおこないます。

これから 2種類のコンテナイメージを作成して Amazon ECR に push します。1つめのコンテナイメージはデータの前処理とバッチ推論で使用し、2つめのコンテナイメージはモデルの学習で使用します。

## データの前処理
データの前処理は Amazon SageMaker Processing の仕組みを使って行います。まずは前処理用のコンテナイメージを作成します。

In [ ]:
ecr_repository = 'gnn-byo-proc'
tag = ':latest'
uri_suffix = 'amazonaws.com'
processing_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

In [ ]:
%%writefile docker/processing/Dockerfile

FROM python:3.8-buster

WORKDIR /opt/app

RUN pip3 install torch==1.8.0

COPY requirements.txt /opt/app
RUN pip3 install -r requirements.txt

RUN pip3 install -U torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html
RUN pip3 install jupyter

COPY . /opt/app

EXPOSE 8888

# jupyter notebook --allow-root --ip=* --no-browser -NotebookApp.token=''


上記 Dockerfile を使ってコンテナイメージをビルドし、Amazon ECR に push します。

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker/processing
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

作成したイメージを使って ScriptProcessor を作成します。このとき、`instance_type` に　`local` を設定するとローカルモードになり、ノートブックインスタンス上で Processing Job が実行されます。作成したコンテナイメージやスクリプトのデバッグをする際は、ローカルモードの利用がおすすめです。デバッグが完了したら、`instance_type` に　インスタンスタイプを設定して Processing Job を実施します。

In [ ]:
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(command=['python3'],
                                   image_uri=processing_repository_uri,
                                   role=role,
                                   sagemaker_session=session,
                                   instance_count=1,
#                                    instance_type='local')
                                   instance_type='ml.c5.xlarge')

Processing Job で使用するスクリプトを作成します。前処理の内容を変更した場合は、前処理スクリプトを更新してから 2つしたのセル（script_processor.run）を再度実行すれば OK です。コンテナイメージの再作成は不要です。

In [ ]:
%%writefile preprocessing.py

import sys
sys.path.append('/opt/app')

import boto3

from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import Planetoid

import torch
import shutil

if __name__=='__main__':
    
    aws_session = boto3.Session(profile_name=None)

    dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
    
    print(f'Dataset: {dataset}:')
    print('======================')
    print(f'Number of graphs: {len(dataset)}')
    print(f'Number of features: {dataset.num_features}')
    print(f'Number of classes: {dataset.num_classes}')
    
    data = dataset[0]  # Get the first graph object.

    print(data)

    # Gather some statistics about the graph.
    print(f'Number of nodes: {data.num_nodes}')
    print(f'Number of edges: {data.num_edges}')
    print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
    print(f'Number of training nodes: {data.train_mask.sum()}')
    print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
    print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
    print(f'Contains self-loops: {data.contains_self_loops()}')
    print(f'Is undirected: {data.is_undirected()}')
    
    # save to container directory for uploading to S3
    
    import os

    path = "./"

    files = os.listdir(path)
    print(files)
    
    
    
    src = 'data/Planetoid/Cora'
    dist = '/opt/ml/processing/output/Cora'
    
    print(os.path.getsize(src))
    
    import tarfile

    # 圧縮
    with tarfile.open('sample.tar.gz', 'w:gz') as t:
        t.add(src)
    
    files = os.listdir(path)
    print(files)
    shutil.copytree(src, dist)
    
    from torch_geometric.io import read_planetoid_data

作成したスクリプトを使って `run` を実行して  Processing Job を起動します。`run` の引数には以下を設定しています。

- code: 処理スクリプトのファイル名
- inputs: （入力データがある場合）入力データが保存されている Amazon S3 パスを `source` に、Processing 用インスタンスのどこに入力データをダウンロードするかを `destination` に設定します。今回はインターネット経由でデータをダウンロードするため使用しません。
- outputs: 出力データを保存する Processing 用インスタンスのパスを `source` で指定し、そこに処理済みのデータなどを保存しておくと、`destination` に設定した S3 パスにそれらのデータが自動的にアップロードされます。
- experiment_config: Processing Job を登録する Experiments があれば、その情報を指定します。

**以下をローカルモードで実行すると、最後に  `PermissionError: [Errno 13] Permission denied: 'ind.cora.tx'` というエラーが出ますが、これはジョブがうまく動いていても出るので無視して構いません。インスタンスを使用した場合はこのエラーは出ません。**

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

processing_job_name = "gnn-byo-process-{}".format(strftime("%d-%H-%M-%S", gmtime()))
output_destination = 's3://{}/{}/data'.format(s3_output, s3_prefix)

script_processor.run(code='preprocessing.py',
                      job_name=processing_job_name,
#                       inputs=[ProcessingInput(
#                         source=raw_s3,
#                         destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(output_name='output',
                                                destination='{}/output'.format(output_destination),
                                                source='/opt/ml/processing/output')],
                      experiment_config={
                            "ExperimentName": gnn_experiment_preprocess.experiment_name,
                            "TrialComponentDisplayName": "Processing",
                      }
                               )

preprocessing_job_description = script_processor.jobs[-1].describe()

## モデルの学習

ここまでで、データの前処理と、前処理済みデータの Amazon S3 へのアップロードが完了しました。次は、前処理済みのデータを使って GNN を学習します。

まずは学習用コンテナイメージを作成します。ベースイメージに、Amazon SageMaker が用意している PyTorch 1.8.0 のイメージを使用しました。

**この Dockerfile はノートブックインスタンスが `us-east-1 (バージニア北部)` の想定なので、他のリージョンをお使いの場合は FROM に書かれている Amazon ECR の URI の `us-east-1` の部分をお使いのリージョンに合わせて書き換えてください。**

In [ ]:
%%writefile docker/train/Dockerfile
# FROM python:3.8-buster
FROM  763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.8.0-cpu-py36-ubuntu18.04

WORKDIR /opt/app

RUN pip3 install torch==1.8.0

COPY requirements.txt /opt/app
RUN pip3 install -r requirements.txt

RUN pip3 install -U torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html
RUN pip3 install jupyter


RUN pip3 install sagemaker-training

WORKDIR /


In [ ]:
ecr_repository = 'gnn-byo-train'
tag = ':latest'
uri_suffix = 'amazonaws.com'
train_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

ベースイメージは Amazon SageMaker が用意している Amazon ECR リポジトリに保存されているため、そこへのアクセス権が必要です。以下のコマンドを実行します。

In [ ]:
!$(aws ecr get-login --region $region --registry-ids 763104351884 --no-include-email)

学習スクリプトを作成します。学習スクリプトの内容を変更した場合は、`pytorch_estimator.fit()` を再度実行すれば OK です。学習スクリプトをコンテナイメージの中に入れておらず、Estimator 経由でコンテナに渡すようにしているため、コンテナイメージの再作成は不要です。

In [ ]:
%%writefile train.py

import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

import json
import argparse
import os


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_features, num_classes):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x



def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

def _save_checkpoint(model, optimizer, epoch, loss, args):
#     print("epoch: {} - loss: {}".format(epoch+1, loss))
    checkpointing_path = args.checkpoint_path + '/checkpoint.pth'
    print("Saving the Checkpoint: {}".format(checkpointing_path))
    torch.save({
        'epoch': epoch+1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        }, checkpointing_path)
    
def _load_checkpoint(model, optimizer, args):
    print("--------------------------------------------")
    print("Checkpoint file found!")
    print("Loading Checkpoint From: {}".format(args.checkpoint_path + '/checkpoint.pth'))
    checkpoint = torch.load(args.checkpoint_path + '/checkpoint.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch_number = checkpoint['epoch']
    loss = checkpoint['loss']
    print("Checkpoint File Loaded - epoch_number: {} - loss: {}".format(epoch_number, loss))
    print('Resuming training from epoch: {}'.format(epoch_number+1))
    print("--------------------------------------------")
    return model, optimizer, epoch_number

if __name__=='__main__':
    parser = argparse.ArgumentParser()

    # Data and model checkpoints directories
    parser.add_argument('--features-num', type=int, default=64, metavar='N',
                        help='input feature size (default: 64)')
    parser.add_argument('--classes-num', type=int, default=1, metavar='N',
                        help='input class size (default: 1)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--backend', type=str, default=None,
                        help='backend for distributed training (tcp, gloo on cpu and gloo, nccl on gpu)')

    # Container environment
    parser.add_argument('--hosts', type=list, default=json.loads(os.environ['SM_HOSTS']))
    parser.add_argument('--current-host', type=str, default=os.environ['SM_CURRENT_HOST'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--data-dir', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--num-gpus', type=int, default=os.environ['SM_NUM_GPUS'])
    parser.add_argument("--checkpoint-path",type=str,default="/opt/ml/checkpoints")

    args = parser.parse_args()

    model = GCN(hidden_channels=16, num_features=args.features_num, num_classes=args.classes_num)
    print(model)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()
    
    path = args.data_dir

    files = os.listdir(path)
    print(files)
    
    from torch_geometric.io import read_planetoid_data
    data = read_planetoid_data(args.data_dir, 'Cora')
    
    # Check if checkpoints exists
    if not os.path.isfile(args.checkpoint_path + '/checkpoint.pth'):
        epoch_number = 0
    else:    
        model, optimizer, epoch_number = _load_checkpoint(model, optimizer, args) 

    for epoch in range(epoch_number, int(args.epochs)+1):
        loss = train()
        acc = test()
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}')
        
        if (epoch %100 == 0):
            _save_checkpoint(model, optimizer, epoch, loss, args)
        
    torch.save(model.state_dict(), args.model_dir+'/model.pth')

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker/train
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $train_repository_uri
!docker push $train_repository_uri

もし、上記コマンドでコンテナイメージをビルドする際に no space left というエラーが出ていたら、以下のコマンドのコメントアウトを解除して実行し、不要なファイルを削除してから再度コンテナイメージのビルドを実行してください。

In [ ]:
# !docker system prune -a -f

Estimator を作成して `fit` で学習ジョブを起動します。ハイパーパラメタの設定や取得したいメトリクスの情報を指定することができます。Processing Job と同様にローカルモードを使用することができます。`fit` の引数には、学習データが保存されている S3 のパスを指定します。PyTorch の Estimator については [こちらのドキュメント](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html#sagemaker.pytorch.estimator.PyTorch) をご参照ください。今回 PyTorch という名前の Estimator を使用しましたが、コンテナイメージの中に学習スクリプトを含めた状態で使用する場合は、Estimator という名前の Estimator を使用してください。

Estimator の `metric_definitions` に記録したいメトリクスの情報を指定することができます。`Regex` には、学習スクリプトが出力するログから取得したい数値を抽出するための正規表現を指定します。つまりメトリクスを記録したい場合は、学習スクリプトがメトリクスに関する情報をログに出力する必要があります。今回は Loss と Acc をメトリクスとして取得するよう設定しています。

Spot Instanceを用いて実行する場合は、下記のコードを Estimator の `instance_type`の次の行あたりに追加します。なお、`max_wait` は、`max_run` 以上の値である必要があります。

```python
max_run = 5000,
use_spot_instances = 'True',
max_wait = 10000,
```

チェックポイントの利用は必須ではありませんが、Spot Instance を使う場合は中断に備えてチェックポイントを有効にすることが推奨されています。チェックポイントの学習インスタンス上の保存パス（checkpoint_local_path）と、それらをアップロードする先のパス（checkpoint_s3_path）を設定し、学習スクリプトにチェックポイントを checkpoint_local_path に保存する記述を追加します。

保存したチェックポイントから学習を再開する場合は、新しく Estimator 定義して引数にチェックポイントが保存してある checkpoint_s3_path と チェックポイントをダウンロードしたいパス checkpoint_local_path を設定して fit を実行します。

チェックポイントの詳細については [こちらのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/model-checkpoints.html#model-checkpoints-enable) をご参照ください。

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.pytorch.estimator import PyTorch
import uuid
import os

# Spot training をする場合は、チェックポイントの設定を推奨
checkpoint_suffix = str(uuid.uuid4())[:8]
checkpoint_s3_path = 's3://{}/checkpoint-{}'.format(bucket, checkpoint_suffix)
checkpoint_local_path="/opt/ml/checkpoints"

pytorch_estimator = PyTorch(
                        entry_point='train.py',
                        image_uri=train_repository_uri,
                        role=role, 
                        instance_count=1,
#                         instance_type='local',
                        instance_type='ml.c4.2xlarge',
                        max_run = 5000,
                        use_spot_instances = 'True',
                        max_wait = 10000,
                        checkpoint_s3_uri=checkpoint_s3_path,
                        checkpoint_local_path=checkpoint_local_path,
                        output_path="s3://{}/output".format(bucket),
                        sagemaker_session=session,
                        hyperparameters = {'epochs': 200, 'features-num':1433, 'classes-num':7, 'lr':0.01},
                        enable_sagemaker_metrics=True,
                        metric_definitions = [dict(
                                                                Name = 'Loss',
                                                                Regex = 'Loss: ([0-9.]+)'
                                                            ),
                                                              dict(
                                                                Name = 'Acc',
                                                                Regex = 'Acc: ([0-9.]+)'
                                                            )
                                             ]
                          
)

pytorch_estimator.fit({'train': os.path.join(output_destination, 'output/Cora/raw/')},
                     experiment_config={
                            "ExperimentName": gnn_experiment_train.experiment_name,
                            "TrialComponentDisplayName": "Training",
                      })

## Amazon SageMaker Experiments でモデルを比較

SageMaker Experiments を使うと複数のモデルのメトリクスなどを比較することができます。上のセルの Estimator の引数で epochs や lr などのハイパーパラメタを変えて何度か学習を実行してから次のセル以降を実行してみましょう。Experiments 内の Trial のフィルタやソートなど方法については [ExperimentAnalytics のドキュメント](https://sagemaker.readthedocs.io/en/stable/api/training/analytics.html#sagemaker.analytics.ExperimentAnalytics) をご参照ください。

メトリクスに関して、DataFrame の列名は Loss - Min などと書かれていますが、ExperimentAnalytics の sort_by で Loss - Min を指定する場合は、metrics.loss.min となります。

In [ ]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=session, 
    experiment_name=gnn_experiment_train.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.acc.max",
    sort_order="Ascending",# Ascending or Descending
    metric_names=['Loss', 'Acc'],
    parameter_names=['epochs', 'lr'],
    input_artifact_names=[]
)

In [ ]:
import pandas as pd
df = trial_component_analytics.dataframe()
pd.set_option('display.max_columns', None)
df

In [ ]:
print(df.columns.tolist())

## Processing Job を使ったバッチ推論

学習したモデルを使ってバッチ推論を行います。今回は、前処理で使用したコンテナイメージを流用してバッチ推論用 Processing Job を起動します。

まずは推論用スクリプトを作成します。<br>
推論結果をグラフにプロットし、その画像を Amazon S3 にアップロードするようにしました。

In [ ]:
%%writefile inference.py

import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

import json
import argparse
import os
import tarfile
import matplotlib.pyplot as plt


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_features, num_classes):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

from sklearn.manifold import TSNE

def visualize(h, color, path):
    z = TSNE(n_components=2).fit_transform(out.detach().cpu().numpy())

    fig = plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
#     plt.show()
    fig.savefig(os.path.join(path, "img.png"))

if __name__=='__main__':
    parser = argparse.ArgumentParser()

    # Data and model checkpoints directories
    parser.add_argument('--features-num', type=str, default='1', metavar='N',
                        help='input feature size (default: 1)')
    parser.add_argument('--classes-num', type=str, default='1', metavar='N',
                        help='input class size (default: 1)')
    parser.add_argument('--model-dir', type=str, default='/opt/ml/model', metavar='N',
                        help='model data path (default: /opt/ml/model)')
    parser.add_argument('--input-dir', type=str, default='/opt/ml/input', metavar='N',
                        help='input data path (default: /opt/ml/input)')
    parser.add_argument('--output-dir', type=str, default='/opt/ml/output', metavar='N',
                        help='output data path (default: /opt/ml/output)')


    args = parser.parse_args()
    
    from torch_geometric.io import read_planetoid_data
    data = read_planetoid_data(args.input_dir, 'Cora')
  
    with tarfile.open(os.path.join(args.model_dir, 'model.tar.gz'), 'r:gz') as t:
        t.extractall()

    model = GCN(hidden_channels=16, num_features=int(args.features_num), num_classes=int(args.classes_num))
    model.load_state_dict(torch.load('model.pth'))
#     print(model)
    
    test_acc = test()
    print(f'Test Accuracy: {test_acc:.4f}')
    
    model.eval()
    out = model(data.x, data.edge_index)
    visualize(out, color=data.y,  path=args.output_dir)


In [ ]:
from sagemaker.processing import ScriptProcessor

batch_inference_processor = ScriptProcessor(command=['python3'],
                                   image_uri=processing_repository_uri,
                                   role=role,
                                   instance_count=1,
#                                    instance_type='local')
                                   instance_type='ml.c5.xlarge')

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

processing_job_name = "gnn-byo-batch-inference-{}".format(strftime("%d-%H-%M-%S", gmtime()))
output_destination_inference = 's3://{}/{}/batch-inference'.format(s3_output, s3_prefix)
input_dir = '/opt/ml/processing/input'
model_dir = '/opt/ml/processing/model'
output_dir = '/opt/ml/processing/output'

model_s3 = pytorch_estimator.model_data
raw_s3 = os.path.join(output_destination, 'output/Cora/raw/')

batch_inference_processor.run(code='inference.py',
                      job_name=processing_job_name,
                      inputs=[ProcessingInput(
                                        source=model_s3,
                                        destination=model_dir),
                                     ProcessingInput(
                                        source=raw_s3,
                                        destination=input_dir)],
                      outputs=[ProcessingOutput(output_name='output',
                                                destination='{}/output'.format(output_destination_inference),
                                                source=output_dir)],
                              arguments=['--model-dir', model_dir, '--input-dir', input_dir, '--output-dir', output_dir , '--features-num', '1433', '--classes-num', '7']
#                       experiment_config={
#                             "ExperimentName": gnn_experiment.experiment_name,
#                             "TrialComponentDisplayName": "Processing",
#                       }
                               )

preprocessing_job_description = batch_inference_processor.jobs[-1].describe()

バッチ推論で出力したプロットの画像をダウンロードして表示します。

In [ ]:
!aws s3 cp $output_destination_inference/output/img.png ./

from IPython.display import Image
Image("./img.png")

## リソースの削除

利用が終わったら、このノートブックを実行したノートブックインスタンスの停止および削除を実施してください。ノートブックインスタンスを停止させると、ノートブックインスタンスの課金は止まりますがアタッチされている EBS ボリュームへの課金が継続しますので、完全に課金を止めるにはノートブックインスタンスの停止だけでなく削除まで実施してください。

また、Amazon S3 にアップロードした各種ファイルに対しても課金が発生するため、不要であれば削除してください。

In [ ]:
sm = boto3.Session().client('sagemaker')
def cleanup(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(sagemaker_boto_client=sm, trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                sagemaker_boto_client=sm,
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
    experiment.delete()

In [ ]:
cleanup(gnn_experiment_preprocess)
cleanup(gnn_experiment_train)